**Pre-defined values from Lunar Time-Transfer paper to define Q and R**

In [5]:
import numpy as np
import math
T_c = 0.00000009775
f_L1 = 1575.42
N_i = 4
CN0 = 15
B_pll = 0.5
T = 20
d = 0.3
pi = math.pi
c = 299792458

B_dll = 0.5

s_dll = (B_dll/(2*CN0))*d*(1+(2/(T*CN0*(2-d))))
pr_var = ((c*T_c)**2)*s_dll

s_pll = (B_pll/(CN0))*(1+(1/(2*T*CN0)))
prr_var = ((c**2)/((2*pi*f_L1 * T_c)**2)) * (s_pll)

R = np.array([[pr_var,0,0,0],[0,pr_var,0,0],[0,0,prr_var,0],[0,0,0,prr_var]])

print(R)


[[4.31066951e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.31066951e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.20521668e+21 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.20521668e+21]]


In [6]:
h_0 = 1.4*(10**(-22))
h_m1 = 2.3*(10**(-26))
h_m2 = 3.3*(10**(-31))

dt = 60

e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*((pi*h_m2*dt)**2)*dt
e2 = h_m1*dt + h_m2*(pi*dt)**2
e3 = (h_0/(2*dt))+ 4*h_m1 + (8/3)*(pi*pi)*h_m2*dt 

Q = np.array([[e1,e2],[e2,e3]])
print(Q)

[[4.36560000e-21 1.39172509e-24]
 [1.39172509e-24 1.25918778e-24]]
